In [47]:
!pip3 install -r requirements.txt

In [48]:
%run logging.ipynb

In [49]:
%run schema.py

In [50]:
%run snowflake_connection.ipynb

In [51]:
%run load_raw_table.ipynb

In [52]:
%run create_table.ipynb

In [53]:
# Importando as bibliotecas
import requests
import zipfile
import io
import os
import pandas as pd
from typing import Optional, Dict
from dotenv import load_dotenv

In [54]:
load_dotenv()

True

In [55]:
# class Logger():
#     def __init__(self):
#         self.format = '%(asctime)s - %(levelname)s - %(message)s'


#     def set_logger(self):
#         logging.basicConfig(format=self.format, level=logging.INFO)
#         logger = logging.getLogger()

#         return logger   

In [56]:
# class SnowflakeHandler(SnowflakeConnector):
#     """
#     Classe para gerenciar a conexão e operações com Snowflake.

#     Atributos:
#     ----------
#     user: str
#         Nome do usuário para a conexão com Snowflake.
#     password: str
#         Senha do usuário para a conexão com Snowflake.
#     account: str
#         Identificador da conta Snowflake.
#     warehouse: str
#         Nome do warehouse a ser usado na conexão.
#     database: str
#         Nome do banco de dados a ser usado na conexão.
#     schema: str
#         Nome do schema a ser usado na conexão.
#     """

#     def __init__(self):
#         """
#         Inicializa a classe SnowflakeHandler carregando as variáveis de ambiente.
#         """
#         super().__init__()
#         self.logger_class = Logger()
#         self.logger = self.logger_class.set_logger()        

#     # def connect(self):
#     #     """
#     #     Estabelece uma conexão com Snowflake usando a configuração fornecida.
        
#     #     Retorna:
#     #     --------
#     #     None
#     #     """

#     #     SNOWFLAKE_CONFIG = {
#     #         'user': self.user,
#     #         'password': self.password,
#     #         'account': self.account,
#     #         'warehouse': self.warehouse,
#     #         'database': self.database,
#     #         'schema': self.schema
#     #     }        
#     #     print(SNOWFLAKE_CONFIG) 
          
#     #     self.session = Session.builder.configs(SNOWFLAKE_CONFIG).create()

#     # def close_connection(self):
#     #     """
#     #     Fecha a conexão com Snowflake.
        
#     #     Retorna:
#     #     --------
#     #     None
#     #     """
#     #     if self.session is not None:
#     #         self.session.close()

#     def save_dataframe(self, df: pd.DataFrame, table_name: str, column_names: list) -> None:
#         """
#         Salva um DataFrame no Snowflake, sobrescrevendo o conteúdo da tabela.

#         Parâmetros:
#         -----------
#         df: pd.DataFrame
#             O DataFrame que será salvo no Snowflake.
#         table_name: str
#             O nome da tabela onde o DataFrame será salvo.
#         column_names: list
#             Lista de nomes das colunas que devem ser usadas no DataFrame.
        
#         Retorna:
#         --------
#         None
#         """
#         super().connect()

#         if not self.connection or not self.cursor:
#             raise RuntimeError("Conexão com Snowflake não estabelecida.")
             
#         df.columns = column_names

#         # Limpar aspas e espaços desnecessários dos nomes das colunas
#         df.columns = df.columns.str.replace(r'^"|"$', '', regex=True).str.strip()

#         # Apagar o conteúdo existente na tabela
#         try:
#             self.cursor.execute(f"DELETE FROM {table_name}")
#             self.logger.info(f"Conteúdo da tabela '{table_name}' deletado com sucesso.")
#         except Exception as e:
#             self.logger.error(f"Erro ao deletar o conteúdo da tabela '{table_name}': {e}")
#             raise

#         # Inserir o novo DataFrame na tabela
#         success, nchunks, nrows, _ = write_pandas(self.connection, df, table_name)

#         if success:
#             self.logger.info(f"DataFrame salvo com sucesso na tabela '{table_name}'. Total de linhas: {nrows}.")
#         else:
#             self.logger.error(f"Falha ao salvar o DataFrame na tabela '{table_name}'.")

#         super().disconnect()


In [57]:
class UrbanMobilityData:
    def __init__(self, url: str):
        """
        Inicializa a classe com a URL do arquivo zip.

        :param url: URL do arquivo zipado do Kaggle.
        """
        self.url = url
        self.file_paths = {
            "Household": "Household.csv",
            "Person": "Person.csv",
            "Stage": "Stage.csv",
            "Trip": "Trip.csv"
        }
        self.logger_class = Logger()
        self.logger = self.logger_class.set_logger()            

    def download_zip(self) -> Optional[bytes]:
        """
        Faz o download do arquivo zipado da URL fornecida.

        :return: Conteúdo do arquivo zipado em bytes, ou None em caso de erro.
        """
        try:
            response = requests.get(self.url)
            if response.status_code == 200:
                return response.content
            else:
                self.logger.error(f"Falha ao fazer o download dos arquivos. Status code: {response.status_code}")
                return None
        except Exception as e:
            self.logger.error(f"Erro ao baixar o arquivo: {e}")
            return None

    def extract_zip_to_dataframes(self, zip_content: bytes) -> Dict[str, Optional[pd.DataFrame]]:
        """
        Extrai o conteúdo do arquivo zipado e carrega os arquivos CSV em DataFrames.

        :param zip_content: Conteúdo do arquivo zipado em bytes.
        :return: Dicionário onde as chaves são os nomes dos arquivos e os valores são os DataFrames ou None.
        """
        dataframes = {}
        try:
            with zipfile.ZipFile(io.BytesIO(zip_content), 'r') as zip_ref:
                for file_name in zip_ref.namelist():
                    if file_name in self.file_paths.values():
                        with zip_ref.open(file_name) as file:
                            df = pd.read_csv(file, sep=';', on_bad_lines='skip')
                            key = [k for k, v in self.file_paths.items() if v == file_name][0]
                            dataframes[key] = df
                            self.logger.info(f"DataFrame {key} carregado com sucesso")
        except Exception as e:
            self.logger.error(f"Erro ao processar o arquivo zipado: {e}")
        return dataframes

In [58]:
if __name__ == "__main__":
    url = os.getenv("URL")
    logger_class = Logger()
    logger = logger_class.set_logger()    
    data_handler = UrbanMobilityData(url)
    create_table = SnowflakeTableCreator()


    # Fazendo o download do arquivo zip
    zip_content = data_handler.download_zip()
    if zip_content:
        # Extraindo o conteúdo do arquivo zip e carregando os DataFrames
        dataframes = data_handler.extract_zip_to_dataframes(zip_content)

        for name, df in dataframes.items():
            if df is not None:
                logger.info(name)

                handler = SnowflakeHandlerRaw()
                # handler.connect()
                # create_table.connect()

                target_table_location = name.upper()

                match name.upper():
                    case 'PERSON':
                        create_table.create_table(target_table_location, table_person)
                    case 'HOUSEHOLD':
                        create_table.create_table(target_table_location, table_household)
                    case 'STAGE':
                        create_table.create_table(target_table_location, table_stage)
                    case 'TRIP':
                        create_table.create_table(target_table_location, table_trip)

                # create_table.disconnect()

                expected_columns = [c.upper() for c in df.columns]

                handler.save_dataframe(df, target_table_location, expected_columns)
        
                # handler.close_connection()

2024-09-07 14:33:56,321 - INFO - DataFrame Household carregado com sucesso
2024-09-07 14:33:56,719 - INFO - DataFrame Person carregado com sucesso
2024-09-07 14:33:57,096 - INFO - DataFrame Stage carregado com sucesso
2024-09-07 14:33:58,314 - INFO - DataFrame Trip carregado com sucesso
2024-09-07 14:33:58,343 - INFO - Household
2024-09-07 14:33:58,347 - INFO - Snowflake Connector for Python Version: 3.10.1, Python Version: 3.10.9, Platform: macOS-10.16-x86_64-i386-64bit
2024-09-07 14:33:58,348 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2024-09-07 14:33:59,558 - INFO - Number of results in first chunk: 1
2024-09-07 14:33:59,560 - INFO - closed
2024-09-07 14:33:59,701 - INFO - No async queries seem to be running, deleting session


Tabela HOUSEHOLD criada com sucesso.


2024-09-07 14:33:59,855 - INFO - Snowflake Connector for Python Version: 3.10.1, Python Version: 3.10.9, Platform: macOS-10.16-x86_64-i386-64bit
2024-09-07 14:33:59,856 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2024-09-07 14:34:01,891 - INFO - Conteúdo da tabela 'HOUSEHOLD' deletado com sucesso.
2024-09-07 14:34:02,168 - INFO - Number of results in first chunk: 1
2024-09-07 14:34:06,684 - INFO - Number of results in first chunk: 1
2024-09-07 14:34:06,686 - INFO - DataFrame salvo com sucesso na tabela 'HOUSEHOLD'. Total de linhas: 19252.
2024-09-07 14:34:06,688 - INFO - closed
2024-09-07 14:34:06,852 - INFO - No async queries seem to be running, deleting session
2024-09-07 14:34:07,037 - INFO - Person
2024-09-07 14:34:07,038 - INFO - Snowflake Connector for Python Version: 3.10.1, Pyt

Tabela PERSON criada com sucesso.


2024-09-07 14:34:08,669 - INFO - No async queries seem to be running, deleting session
2024-09-07 14:34:08,979 - INFO - Snowflake Connector for Python Version: 3.10.1, Python Version: 3.10.9, Platform: macOS-10.16-x86_64-i386-64bit
2024-09-07 14:34:08,980 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2024-09-07 14:34:10,452 - INFO - Conteúdo da tabela 'PERSON' deletado com sucesso.
2024-09-07 14:34:10,774 - INFO - Number of results in first chunk: 1
2024-09-07 14:34:14,610 - INFO - Number of results in first chunk: 1
2024-09-07 14:34:14,611 - INFO - DataFrame salvo com sucesso na tabela 'PERSON'. Total de linhas: 61358.
2024-09-07 14:34:14,611 - INFO - closed
2024-09-07 14:34:14,813 - INFO - No async queries seem to be running, deleting session
2024-09-07 14:34:14,999 - INFO - Stage
2024

Tabela STAGE criada com sucesso.


2024-09-07 14:34:16,481 - INFO - Snowflake Connector for Python Version: 3.10.1, Python Version: 3.10.9, Platform: macOS-10.16-x86_64-i386-64bit
2024-09-07 14:34:16,483 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2024-09-07 14:34:17,812 - INFO - Conteúdo da tabela 'STAGE' deletado com sucesso.
2024-09-07 14:34:18,111 - INFO - Number of results in first chunk: 1
2024-09-07 14:34:22,641 - INFO - Number of results in first chunk: 1
2024-09-07 14:34:22,643 - INFO - DataFrame salvo com sucesso na tabela 'STAGE'. Total de linhas: 117730.
2024-09-07 14:34:22,646 - INFO - closed
2024-09-07 14:34:22,809 - INFO - No async queries seem to be running, deleting session
2024-09-07 14:34:22,970 - INFO - Trip
2024-09-07 14:34:22,971 - INFO - Snowflake Connector for Python Version: 3.10.1, Python Versi

Tabela TRIP criada com sucesso.


2024-09-07 14:34:25,061 - INFO - Snowflake Connector for Python Version: 3.10.1, Python Version: 3.10.9, Platform: macOS-10.16-x86_64-i386-64bit
2024-09-07 14:34:25,062 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2024-09-07 14:34:27,026 - INFO - Conteúdo da tabela 'TRIP' deletado com sucesso.
2024-09-07 14:34:27,420 - INFO - Number of results in first chunk: 1
2024-09-07 14:34:36,453 - INFO - Number of results in first chunk: 1
2024-09-07 14:34:36,454 - INFO - DataFrame salvo com sucesso na tabela 'TRIP'. Total de linhas: 113398.
2024-09-07 14:34:36,455 - INFO - closed
2024-09-07 14:34:36,624 - INFO - No async queries seem to be running, deleting session
